# MAC0209 - Modelagem e Simulação

Pynamical: Demo of logistic map and bifurcation diagrams

Author: Geoff Boeing

Adapted and modified by: R. Hirata Jr., Artur André, Roberto. M. Cesar Jr to be used in MAC0209

Leia o artigo completo aqui:
http://geoffboeing.com/2015/03/chaos-theory-logistic-map/

## MAC0209 - Modelagem e Simulação

In [ ]:
!python -m pip install pynamical==0.3.1
!python -m pip install imageio

In [ ]:
from pynamical import logistic_map, simulate, bifurcation_plot
import pynamical
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
plt.rcParams.update({'font.size': 22})

print(f"pynamical version: {pynamical.__version__}")
print(f"matplotlib version: {matplotlib.__version__}")
print(f"numpy version: {np.__version__}")


# Apenas para recapitular, aqui temos o diagrama de bifurcação, para o mapa logístico, gerado pelo pacote pynamical.

In [ ]:
pops = simulate(model=logistic_map, num_gens=100, rate_min=0, rate_max=4, num_rates=1000, num_discard=100)
bifurcation_plot(pops)

In [ ]:
xini = 0.3
r = 3.429
x = [xini]
for _ in range(100):
    x.append(logistic_map(x[-1], r))

plt.figure(figsize=(15,3))
plt.plot(list(range(0,len(x))), x[0:])
plt.title("r = 3.429");
plt.xlabel('Geração')
plt.ylabel('População')
print(f"Últimos valores de população:")

# Ao usarmos uma taxa de crescimento $r = 3.429$ observamos um comportamento periódico (com 2 valores) da população $x$ ao longo das iterações do mapa logístico após uma certa iteração.

In [ ]:
xini = 0.5
r = 3.429
x = [logistic_map(xini, r)]
for _ in range(10000):
    x.append(logistic_map(x[-1], r))

plt.figure(figsize=(15,3))
plt.scatter(list(range(9900,len(x))), x[9900:])
plt.title("r = 3.429");
plt.xlabel('Geração')
plt.ylabel('População')
print(f"Últimos valores de população:")
print(f"x[9996] = {x[9996]}")
print(f"x[9997] = {x[9997]}")
print(f"x[9998] = {x[9998]}")
print(f"x[9999] = {x[9999]}")

# Exercícios

Consulte a descrição dos exercícios nos slides da aula sobre Caos.

## Sistema de EDOs acopladas

Para solucionar as equações de Lorenz (nos exercícios) podemos usar o método de Euler também. No caso precisaremos resolver um sistema de equações da forma:



$\frac{dx}{dt} = xy - x$
$\frac{dy}{dt} = y - xy + sin^2(\omega t)$

- As equações acima foram baseadas nesta [apresentação](https://www.if.ufrj.br/~sandra/MetComp/2020-1e/Aula11/Aula11.pdf) da Profa. Dra. Sandra Amato.

Abaixo temos uma possivel implementação do método de Euler para resolver esse sistema de EDOs acopladas:

In [ ]:
# funcoes genericas que podem ser re-usadas em outros problemas

import math
import matplotlib.pyplot as pyplot
import numpy as np
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D


# funcoes base para implementar o Euler. 
# Deve-se implementar a funcao rates, que depende de cada modelo.

def initStateVector(s):
    return np.array(s)

def updateStateVectorEuler(s,dt):
    return s + rates(s,dt)

# State Vector Trajectories store state space evolution. Uses list to init empty.

def initSVTrajectory():
    return []

# append s a svt
def updateSVTrajectory(svt,s):
    svt.append(s)
    return svt

def extractSVTrajectory(svt,i): # returns the trajectory as numpy array
    foo = np.array(svt)
    return foo[:,i]
  
def plotEuler(vxe, vtime):
    fig, ax = pyplot.subplots()
    pyplot.plot(vtime, vxe, label='Euler',linestyle='',marker='o') 
    pyplot.title('Posição')
    ax.set_xlabel('Tempo (segundos)')
    ax.set_ylabel('Posição (metros)')
    pyplot.show(block=False)
    
def erroTrajetorias(v1,v2,tipoErro):
    if (tipoErro == 0): # erro com sinal
        return(np.array(v1) - np.array(v2))
    elif (tipoErro == 1): # erro quadratico
        return((np.array(v1) - np.array(v2))**2)
    elif (tipoErro == 2): # erro em modulo
        return(fabs((np.array(v1) - np.array(v2))))
    

def easyPlot(v,title):
    pyplot.figure()
    pyplot.plot(v)
    pyplot.title(title)
    pyplot.show()

def easyPlot2D(x,y,title):
    pyplot.figure()
    pyplot.plot(x,y,'*')
    pyplot.title(title)
    pyplot.show()
    
def easyPlot3D(x,y,z,title,xl,yl,zl):
    mpl.rcParams['legend.fontsize'] = 10
    fig = pyplot.figure()
    ax = fig.gca(projection='3d')
    ax.plot(x, y, z, label=title)
    ax.set_xlabel(xl)
    ax.set_ylabel(yl)
    ax.set_zlabel(zl)
    ax.legend()
    pyplot.show()


In [ ]:
# Euler:
def rates(s,dt):
    omega = 0.115
    x1 = dt*(s[0]*s[1]-s[0]) # +x0
    y1 = dt*(s[1]-s[0]*s[1] + np.sin(omega*s[2])**2) # +y0
    r0 = x1 
    r1 = y1 
    r2 = dt 
    return np.array([r0,r1,r2])

def main():
    t=0
    tf = 10
    dt = 0.1
    x0 = 1.2
    y0 = 3.1
    
    # state vector: [position x, position y, time t]
    stateVectorEuler = initStateVector([x0,y0,t])
    
    
    svtEuler = initSVTrajectory() 
    
    while (stateVectorEuler[2] < tf):    
        svtvEuler        = updateSVTrajectory(svtEuler,list(stateVectorEuler))
        stateVectorEuler = updateStateVectorEuler(stateVectorEuler,dt)
        #print(stateVectorEuler)
        #break

    vx = extractSVTrajectory(svtEuler,0)
    vy = extractSVTrajectory(svtEuler,1)

    vtime = extractSVTrajectory(svtEuler,2)

    easyPlot2D(vtime, vx, 't, x')
    easyPlot2D(vtime, vy, 't, y')
    easyPlot2D(vx, vy, 'x, y')

 

main() 


## Mapa logístico / Diagrama de bifurcação / Cobweb

1. Gere um gráfico de cobweb (figura c) no quadro branco) para o mapa logístico. Dica: Consulte a página do [pynamical](https://github.com/gboeing/pynamical)
2. Agora usando o `matplotlib.pyplot.scatter` gere o diagrama de bifurcação apenas para um subconjunto de gerações de $x_{min}$ até $x_{max}$ depois convergência ao ponto fixo,	eg. $x_{9000}$ até $x_{10000}$. Por exemplo, para $r = 3.429$ após 9996 iterações notamos que $x_n$ onde $n \in [9996, ...]$, basicamente assume os valores $0.8468095286369081$ e $0.44482068425314786$, ou seja, para este $r$ o mapa de bifurcação teria 2 pontos no eixo vertical.



## Equações de Lorenz

1. Usando o método de Euler resolva numéricamente as equações de Lorenz:
    - $\frac{dx}{dt} = \sigma(y-x)$
    - $\frac{dy}{dt} = x(\rho - z) - z$
    - $\frac{dz}{dt} = xy - \beta z$
    
2. Gere os 4 tipos de gráficos (ilustrados no quadro branco à direita a) b) c) e d)):
    - a) $x(t), y(t), z(t)$
    - b) $x \times y$, $x \times z$, $y \times z$
    - c) $x \times y \times z$
    - d) $x$, $x \times t$, $x$, $z \times t$, $y$, $z \times t$

### Como gabarito, vamos escolher alguns valores para as constantes $\sigma = 10, \rho = 28$ e $\beta = 8/3$.
### como condições de contorno: $x_0 = 0, y_0 = 1$ e $z_0 = 0$
### e vamos iterar do tempo $t = 0$ até $t = 50$ com um passo $dt = 0.01$.

### Para esta situação obtemos os gráficos abaixo para $x \times t$ e para $z \times x$:

## Mapa de Lorenz, dois tipos de gráficos:

    - a) x(n), y(n), x X y
    - b) Mapa de bifurcações como o mapa logístico